In [1]:
# Set up GRASS environment
import os
import csv
from IPython.display import Image
from helper import show_interactively, show
import grass.script as gs
# had to add shell = True to helper.py for this to work
from helper import initialize_GRASS_notebook
initialize_GRASS_notebook(binary='grass83',
                          grassdata="D:\\grassdata", location="FUTURES_zoning_prelim",
                          mapset="input")

In [2]:
import validation

In [3]:
# change to mapset to where appropriate results are (core, sub, or base)
!g.mapset mapset=core
# !g.region vector=central_pines_regional_council@input res=30
# !r.mask raster=sa_counties@input

Mapset switched. Your shell continues to use the history for the old mapset


In [3]:
# check if mask is in place
!g.list mapset=core type=raster pattern="MASK"

MASK


In [9]:
%%writefile reclass.txt
-1 0 = 0
1 thru 100 = 1
* = 0

Writing reclass.txt


In [ ]:
# calculate probabilty rasters for each scenario

In [ ]:
for seed in range(1, 51):
    gs.run_command(
        "r.reclass",
        input=f"out_seed_{seed}",
        output=f"out_seed_{seed}_binary",
        rules="reclass.txt",
    )
gs.run_command(
    "r.series",
    input=[f"out_seed_{seed}_binary" for seed in range(1, 11)],
    output="probability",
    method="sum",
    weights=[0.1] * 10,
    nprocs=2,
)

In [9]:
import json
metrics = []
n = 1
# !r.mask masking_2019  --o
for i in range(1, n + 1):
    !r.reclass input=final_run{i} output=final_run{i}_reclass rules=reclass.txt --o
    metrics.append(json.loads(gs.read_command('r.futures.validation', simulated=f'final_run{i}_reclass', reference='urban_2021@input', original='urban_2001@input', format='json')))
# !r.mask -r

In [14]:
metrics

[{'quantity_class_0': 0.22292079064897494,
  'quantity_class_1': 0.22292079064897494,
  'allocation_class_0': 0.034559860049433,
  'allocation_class_1': 0.034559860049433,
  'total_quantity': 0.22292079064897494,
  'total_allocation': 0.034559860049433,
  'kappa': -0.024984143305588388,
  'kappasimulation': 0.003056185472552455,
  'misses': 0.04032465184208141,
  'hits': 0.0013662315460612503,
  'false_alarms': 0.0172799300247165,
  'null_successes': 0.7411507268005136,
  'figure_of_merit': 0.02316792777634182,
  'producer': 0.03277051084146184,
  'user': 0.07327146345242484,
  'initially_developed': 0.1998784597866272},
 {'quantity_class_0': 0.22287954667492874,
  'quantity_class_1': 0.22287954667492874,
  'allocation_class_0': 0.03462302111113657,
  'allocation_class_1': 0.03462302111113657,
  'total_quantity': 0.22287954667492874,
  'total_allocation': 0.03462302111113657,
  'kappa': -0.02498440949460031,
  'kappasimulation': 0.003113930273852339,
  'misses': 0.04031498839888701,
  

In [13]:
# to get hits, misses, etc.
import json
metrics = []
n = 50
# !r.mask masking_2019  --o
for i in range(1, n + 1):
    !r.reclass input=final_run{i} output=final_run{i}_reclass rules=reclass.txt --o
    cats, results = validation.compute('urban_2021@input', f'final_run{i}_reclass', 'urban_2001@input')
    metrics.append(validation.print_results('json', cats, **results))
# 

In [15]:
# save metrics to pickle file
import pickle
with open("metrics", "wb") as pf:
  pickle.dump(metrics, pf)

In [ ]:
# get info about developed cells per zone

In [ ]:
# can do for each simulation or just for the >50% probability?